<H1> AmorphousMLAlone.ipynb </H1>

**Warning. When loading a single experiment and Hot-encoding CellID the program won't know it it should encode one or three values. (brings up an error of shape mismatch)
WARNING VERY VERY IMPORTANT, IF A NEW CELL IS ADDED, TWO EXPERIMENTS WITH THAT CELL ARE REQUIRED OR HOT ENCODING WILL BREAK (AND THE PROGRAM WON'T PREDICT WITH ZERO EXAMPLES OF THAT CELL)**

This code requires that AmorphousFileLectureTests.ipynb has been run. As the data base is extremely small, the effects of overfitting and underfitting are very prone to happening. To test if a model performs as expected it should try to predict files that have not been used in the process of training and validation. As we can't extract too many files form the data base, the solution is to extract only one experiment, train a model with a given structure with all the other experiments, try to predict this isolated experiment (it is a blind prediction where we know what the output looks like) and repeat this entire process for all experiments. To compare the results of different models please use the code files inside LTestVisualCheck

__________________________________________________________________________________________

OUTPUTS OF THE CODE: 

1. **AmorphousLog\_Testing\_ML.txt**
A log file with every step that the algorithm has followed


2. **AmorphousExecution\_times.txt**
It times how long the code took to loop for each model to loop over all the isolation 


3. **AmorphousAllTestsFolder\_{Complexity}\_{num_augmentations}** 
For each Complexity and each num_augmentations (a.k.a, for each model structure and data organization) a folder is created. Inside there are folders for each isolated experiment iteration

    3.1 Missing{base\_name}
For each experiment in the data base (AmorphousMLDataBase) a folder is created and inside there are all the files that the code has created when training and predicting
 
        3.1.1 Difference\_{base\_name}.jpg
For each experimental point, we obtain the prediction and we obtain the difference between prediction and experimental values. It may be helpful to detect an overall deficiency when predicting

        3.1.2 Difference\_{base\_name}.txt
Has the same information as the last .jpg but written on a .txt file

        3.1.3 Missing\_{base\_name}.jpg
This is the important graph. It shows in black the experimental values with their uncertainty, in red the pure ML predictions (without the correction) with the uncertainty bands in a transparent red, in green the final predictions (ML + linear correction) with uncertainty as the green transparent band and in blue the corrected predictions for the time points where experimental values are known

        3.1.4 PredictedData\_PolarizationD3\_Missing{base\_name}.txt
The green part of the graph but in a .txt file

        3.1.5 PredictedPoints\_PolarizationD3\_Missing{base\_name}.txt
The blue part of the graph but in a .txt file

        3.1.6 RawData\_PolarizationD3\_Missing{base\_name}.txt
The black part of the graph but in a .txt file. It is supposed to be a duplicate of the file in AmorphousMLDataBase but it is stored to make sure that the scaling process is being done accurately (if it doesn't fit, be very very careful)

        3.1.7 model_PolarizationD3.keras
The model used in this isolated-experiment iteration

        3.1.8 scaler_static_PolarizationD3.pkl
The scaler for the static parameters (initial and final polarizations included) used in this isolated-experiment iteration

        3.1.9 scaler_time_PolarizationD3.pkl
The scaler for the time evolution used in this isolated-experiment iteration

        3.1.10 scaler_y_PolarizationD3.pkl
The scaler for the polarization values used in this isolated-experiment iteration


Here are all funtions and a overall guide of what the code does:

    1. Define the model and decide the number of augmentations
    1. Creates the folder where the isolated files will temporally reside
    2. Loops for all pairs of array and parameter files
    3. Moves both, parameter and array, files to the isolated directory
    4. A subfolder on the main folder gets created with the name of the isolated experiment. All results will be stored here for this isolation iteration.
    5. Loop for all polarization modes to be processed. It can be SoftPolarizationD3, PolarizationD3 or both
    6. Load the rest of the files and augment them
    7. Obtain the time and polarization scalers. They are obtained using all time values and all polarization values including the soon-to-become static features
    8. Obtain the static features scaler.
    9. Save scalers locally
    10. Train the model. The data gets scaled during this step.
    11. Load the isolated experiment and align both sets of data (use align_static_vectors)
    12. Build manually the dataset for the isolated experiment
    13. Extract the initial and final points, add them as parameters and scale the entire static vector with the general scaler.
    14. Scale time and polarization arrays (only intermediate points). This is compatible with the scaling done to the rest of the experiments.
    15. Combine all structures so that the shape is correct for prediction.
    16. Check for shapes and create the final NumPy arrays of the experiment
    17. Extract the initial and final polarizations and keep a scaled and unscaled version of all. Then predict at those time points. 
    18. Prepare the linear correction and predict in the time points where we have data. The unscaled (physical units) outputs are pred_polar and pred_sigma.
    19. Plot the raw values (black), the predictions for those time points (blue) and predictions every 1000 seconds (red if there is no correction, green if there is correction)
    20. Save data
    21. Move all experiments back to the original folder


1. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorithm. The output is as follows:
    
    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    Note that the Cell Ids (polariser and analyser) are both Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to the Cell_IDs. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical varaibles to ML.
    The size of the output list depends on the number of pairs of .tx files present in the folder. This means that it also works for isolated experiments
    
2. **augment_experiments** is a function that takes the original data list (as load_experiments returns it) and augments them _num\_augmentations_ times. The final legth of the list will be (num_augmentations +1 ) times the original length. Due to the reduced size of the data base (Not more than 30 experiments before 2026), techniques like augmentation are required. Due to the stochastic nature of neutron detection, most measurements, if repeated under the same conditions, will yield different results. Of course, all measurements converge (with uncertainty) to what we can call "the true value". Therefore, we can duplicate the experiments adding noise to the measurements to obtain "hypothetical" measurements that expand the data base. It was decided that the uncertainty won't be modified. To decide what type of noise could be applied two possibilities were considered. The first one was to suppose that sensor detection foloows a Poissonian distribution (law of rare events). The second one was to suppose that it follows a Normal Distribution of mean the measured value and width the uncertainty. As a poissonian distribution converges stochastically (in probability and distribution but not almost sure nor in L^p) to a normal distribution, it is safe to assume that they converge to the same result so the gaussian approach was selected. Also, the measurements are not raw counts but normalized values. Therefore the
    uncertainty is not the square root of the mean. As, we hope, that polarization measurements were done with more than 50 counts, the error made should be negligible.  

3. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move to place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs
(over-fitting) and worsen any new predicitions. 
Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too
much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method  that of the augmentation of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to duplicate these values. For the augmented 
experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we
have just said as these work as "hypothetical independent experiments". This is why sugmentation is done before this function gets used.
         

4. **nll_loss** ML algorithms require a way to tell the algorithm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorithm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL $=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance: $Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).
$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.
Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit.
Therefore, an addition penalty was added.
$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty

5. **Define\_Complexity**. Given the name of a model it defines the model function of the ML algorithm. To be precise, it defines a function called _build_model_ every time _Define\_Complexity_ runs. If _Define\_Complexity_ gets run another time, it will define (possibly) another _build_model_ if the _Complexity_ variable changes. To find the best model, first two models were obtaines, one that overfits and another one that underfits. Then a series of intermediate models are built and tested. As ML model training has an important random aspect, there is no continuous transition from models with similar characteristics. Therefore, 185 models were tested whose names represent the main feature that defined them (NaifTwice1D have two Conv1D layers, SimpleNoDroput was the Simple model without the Dropout layer, etc). The numbers represent the size of the layers or a property of those layers (num_kernel for example).
                Complex, Simple, Naif, Naif834, Naif838, Naif8316, Naif8332, Naif854, Naif858, Naif8516,
                Naif8532, Naif8104, Naif8108, Naif81016, Naif81032, Naif1634, Naif1638, Naif16316, Naif16332,
                Naif1654, Naif1658, Naif16516, Naif16532, Naif16104, Naif16108, Naif161016, Naif161032, Naif2434,
                Naif2438, Naif24316, Naif24332, Naif2454, Naif2458, Naif24516, Naif24532, Naif24104, Naif24108,
                Naif241016, Naif241032, NaifTwice1D883316, NaifTwice1D883332, NaifTwice1D883516, NaifTwice1D883532,
                NaifTwice1D885316, NaifTwice1D885332, NaifTwice1D885516, NaifTwice1D885532, NaifTwice1D843316,
                NaifTwice1D843332, NaifTwice1D843516, NaifTwice1D843532, NaifTwice1D845316, NaifTwice1D845332
                NaifTwice1D845516, NaifTwice1D845532, NaifTwice1D483316, NaifTwice1D483332, NaifTwice1D483516,
                NaifTwice1D483532, NaifTwice1D485316, NaifTwice1D485332, NaifTwice1D485516, NaifTwice1D485532,
                NaifTwice1D443316, NaifTwice1D443332, NaifTwice1D443516, NaifTwice1D443532, NaifTwice1D445316,
                NaifTwice1D445332, NaifTwice1D445516, NaifTwice1D445532, NaifTwiceDense414, NaifTwiceDense418
                NaifTwiceDense4116, NaifTwiceDense4124, NaifTwiceDense814, NaifTwiceDense818, NaifTwiceDense8116,
                NaifTwiceDense8124, NaifTwiceDense1614, NaifTwiceDense1618, NaifTwiceDense16116, NaifTwiceDense16124,
                NaifTwiceDense2414, NaifTwiceDense2418, NaifTwiceDense24116, NaifTwiceDense24124, NaifTwiceDense424,
                NaifTwiceDense428, NaifTwiceDense4216, NaifTwiceDense4224, NaifTwiceDense824, NaifTwiceDense828,
                NaifTwiceDense8216, NaifTwiceDense8224, NaifTwiceDense1624, NaifTwiceDense1628, NaifTwiceDense16216,
                NaifTwiceDense16224, NaifTwiceDense2424, NaifTwiceDense2428, NaifTwiceDense24216, NaifTwiceDense24224,
                SimpleNoDropout444, SimpleNoDropout448, SimpleNoDropout484, SimpleNoDropout488, SimpleNoDropout4164,
                SimpleNoDropout4168, SimpleNoDropout844, SimpleNoDropout848, SimpleNoDropout884, SimpleNoDropout888,
                SimpleNoDropout8164, SimpleNoDropout8168, SimpleNoDropout1644, SimpleNoDropout1648, SimpleNoDropout1684,
                SimpleNoDropout1688, SimpleNoDropout16164, SimpleNoDropout16168, Simple414414, Simple414418,
                Simple414424, Simple414428, Simple414814, Simple414818, Simple414824, Simple414828, Simple418414,
                Simple418418, Simple418424, Simple418428, Simple418814, Simple418818, Simple418824, Simple418828,
                Simple424414, Simple424418, Simple424424, Simple424428, Simple424814, Simple424818, Simple424824,
                Simple424828, Simple428414, Simple428418, Simple428424, Simple428428, Simple428814, Simple428818,
                Simple428824, Simple428828, Simple814414, Simple814418, Simple814424, Simple814428, Simple814814,
                Simple814818, Simple814824, Simple814828, Simple818414, Simple818418, Simple818424, Simple818428,
                Simple818814, Simple818818, Simple818824, Simple818828, Simple824414, Simple824418, Simple824424,
                Simple824428, Simple824814, Simple824818, Simple824824, Simple824828, Simple828414, Simple828418,
                Simple828424, Simple828428, Simple828814, Simple828818, Simple828824, Simple828828



6. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    6.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation. However this has not been tested. A set of models should be ran with and without validation to check if it affects positively, negatively or if it doesn't matter. **Possible optimization of the algorithm**
    
    6.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    6.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for amorphous materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.

7. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

8. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

    8.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
    8.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
    8.3. It builds the model depending on the use_uncertainty bool. (It changes the loss function and the output).
    
    8.4. It trains the model and returns its history (the trained model)
    
9. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
10. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the time points stored as static features are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still universal. However we can't fully say that it is a pure ML algorithm. The 'correctness' of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.

11. **isolated_experiments**. It does the entire Leave-One-Out logic and uses all the previous functions


## 1- Libraries

In [8]:
import os
import psutil
import glob
import math
import shutil
import gc
import joblib
import sys
import time
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras import Model, Input, regularizers, backend as K
from tensorflow.keras.layers import (Dense, Dropout, Concatenate, Conv1D, GlobalAveragePooling1D)
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)
from tensorflow.keras.optimizers import Adam


## 2. Auxiliary Functions and log file creation

1. _PrintDebug_ is a flag that allows the code to output on screen all the steps. If it is set to false, it won´t show anything. However, all information will be properly logged whether this flag is set to true or false. The name of the log is determined by the variable *log_file_path*. The code runs faster if it is set to False.

2. _ShowPlot_ is a similar flag that allows the code to show on screen all plots that are being produced. They are all stored independently of whether this flag is True or False. The code runs faster if it is set to False.

3. _LogNoise_ is a flag that can toggle on or off if numerical values get written in the log. These values are not very useful but, if there is ever a need to check what is going on numerically, this flag can help.

3. **log_message** is a function used for writting on the log file. If you run this cell ti will first erase the previous log. If you need to keep information of what happened please save it manually. It also erases the elapsed time that each model has taken.

4. **long_path** is a function that "fixes" directory paths

In [2]:
PrintDebug = False
ShowPlot = False 
LogNoise = False 


log_file_path = "AmorphousLog_Testing_ML.txt"
def log_message(message):
    """
    Arguments: 
        message (string): The text that will be logged
    
    Returns:
        None
        
    Notes:
        It will write the string "message" in the log file.
        If PrintDebug==True then it will also print the string
    """
    message = str(message)
    if PrintDebug:
        print(message)
    with open(log_file_path, 'a', encoding='utf-8') as log_file:
        log_file.write(str(message) + "\n")

################################################################

def long_path(path):
    """
    Arguments: 
        path (str): The path string that needs to be converted
    
    Returns:
        The updated path string
        
    Notes:
        To avoid Windows 260 character limit for paths, a special "prefix" is added.
        It also unifies how directories are managed.

    """
    # Convert to Path and resolve to absolute
    path = Path(path).resolve()

    # Convert to string
    path_str = str(path)

    # Prepend \\?\ if not already present
    if not path_str.startswith("\\\\?\\"):
        path_str = "\\\\?\\" + path_str

    return path_str


to_erase = ["AmorphousExecution_times.txt", "AmorphousLog_Testing_ML.txt"]
for item in to_erase:
    path = os.path.abspath(item)  # full path
    if os.path.exists(path):
        try:
            if os.path.isfile(path):
                os.remove(path)
                log_message(f"Deleted file: {path}")
            elif os.path.isdir(path):
                shutil.rmtree(path)
                log_message(f"Deleted folder: {path}")
        except Exception as e:
            log_message(f" Could not delete {path}: {e}")
    else:
        log_message(f"Not found (skipped): {path}")
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_file.write("=== Log started ===\n")
    log_file.write("All outputs from functions have a string at the beginning to show the origin:\n")

    
    


## 3. Functions

1. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorithm. The output is as follows:
    
    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    Note that the Cell Ids (polariser and analyser) are both Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to the Cell_IDs. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical variables to ML.
    The size of the output list depends on the number of pairs of .txt files present in the folder. This means that it also works for isolated experiments
    
2. **augment_experiments** is a function that takes the original data list (as load_experiments returns it) and augments them _num\_augmentations_ times. The final legth of the list will be (num_augmentations +1 ) times the original length. Due to the reduced size of the data base (Not more than 30 experiments before 2026), techniques like augmentation are required. Due to the stochastic nature of neutron detection, most measurements, if repeated under the same conditions, will yield different results. Of course, all measurements converge (with uncertainty) to what we can call "the true value". Therefore, we can duplicate the experiments adding noise to the measurements to obtain "hypothetical" measurements that expand the data base. It was decided that the uncertainty won't be modified. To decide what type of noise could be applied two possibilities were considered. The first one was to suppose that sensor detection foloows a Poissonian distribution (law of rare events). The second one was to suppose that it follows a Normal Distribution of mean the measured value and width the uncertainty. As a poissonian distribution converges stochastically (in probability and distribution but not almost sure nor in L^p) to a normal distribution, it is safe to assume that they converge to the same result so the gaussian approach was selected. Also, the measurements are not raw counts but normalized values. Therefore the
    uncertainty is not the square root of the mean. As, we hope, that polarization measurements were done with more than 50 counts, the error made should be negligible.  

3. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move to place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs
(over-fitting) and worsen any new predicitions. 
Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too
much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method  that of the augmentation of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to duplicate these values. For the augmented 
experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we
have just said as these work as "hypothetical independent experiments". This is why sugmentation is done before this function gets used.
         

4. **nll_loss** ML algorithms require a way to tell the algorithm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorithm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL $=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance: $Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).
$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.
Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit.
Therefore, an addition penalty was added.
$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty


In [3]:
def load_experiments(data_dir, polarization_column):
    """
    Arguments:
        data_dir (str): The direction to the folder where all files to be loaded will be found
        polarization_column: It is the name in the header (in the .txt files) of the column that will be read.
                             It can be 'SoftPolarizationD3' or 'PolarizationD3'. All results were obtained with 'PolarizationD3'
    Output: 
        A list, for all polarization decays like:
        encoded_experiments = [
            (static_values,   # list of static parameters (per experiment)
            Deltatime,       # 1D numpy array of time values
            polarization,    # 1D numpy array of polarization values
            Uncertainty      # 1D numpy array of uncertainty values), ...]
        A pd object with the Hot encoded Ids and the rest of the parameters per experiment (each experiment in a different row)
    
    Notes:
    The steps the code does are the following:
    1. Finds all array files (the ones with the numerical values of the decay) and loops over all of them
    2. For each array files it reconstructs the name of the parameter file. It concatenates all parameter files into one pd structure.
    3. Finally it loops over all array files appending the parameters, time, polarization and uncertainty of every experiment to a common list
    """
    log_message(f"    load_experiments: Finding all Array Files...")
    
    # Step 1:
    arrays_files = sorted(
        glob.glob(os.path.join(data_dir, "*.txt"))) #Find all files that are .txt
    arrays_files = [f for f in arrays_files if not f.endswith("_Parameters.txt")] #Keep only the Arrays (not the parameters) (we are only working with the names!)

    encoded_experiments = []
    all_static_df = []

    static_columns = ['CellID','AnalyserID', 'PolariserPressure','AnalyserPressure', 'LabPolarization', 'LabTime'] #Parameter header

    for arrays_path in arrays_files:
        # Step 2:
        base = os.path.basename(arrays_path)
        # Build parameters filename by adding _Parameters before .txt
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)

        # Read parameters file
        try:
            parameters_df = pd.read_csv(parameters_path) #Import the parameter file
            if LogNoise:
                log_message(f"    load_experiments: Reading parameters file: {parameters_filename}") #Clutter logging

            #Get the first row as static data
            static_row = parameters_df.iloc[0][static_columns] #Get the parameter numerical values
            all_static_df.append(static_row)

        except Exception as e:
            log_message(f"    ****load_experiments: Failed to read parameters file: {parameters_filename}, error: {e}")
            continue

    log_message(f"    load_experiments: Create combined DataFrame for static parameters...")
    static_df = pd.DataFrame(all_static_df) #Combine all static rows into a Dataframe

    log_message(f"    load_experiments: Collected static data:")
    
    
    """
    The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to the Cell_IDs.
    As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes
    0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical varaibles to ML.
    """
    
    log_message(f"    load_experiments: Static dataframe columns:, {static_df.columns.tolist()}")
    log_message(f"    load_experiments: Static dataframe shape:, {static_df.shape}")

    log_message(f"    load_experiments: Hot encoding CellID.") 
    categorical_cols = ['CellID', 'AnalyserID']
    static_df = pd.get_dummies(static_df, columns=categorical_cols, prefix=['CellID', 'AnalyserID'])
    
    # Now second pass: read arrays and create encoded_experiments with encoded static params
    for i, arrays_path in enumerate(arrays_files):
        base = os.path.basename(arrays_path)
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)
        # log_message(f"Reading arrays file: {base}") #Clutter log
        arrays_df = pd.read_csv(arrays_path) # Reads the time series data 
    
        static_values = static_df.iloc[i].to_list() #Fetches the static parameters corresponding to this experiment
    
        Deltatime = arrays_df["DeltaTime"].values
        polarization = arrays_df[polarization_column].values #Extracts the time array and the selected polarization column.
        #Save the uncertainty even if it is not used afterwards
        Uncertainty = arrays_df["ErrPolarizationD3"].values
        #if len(Deltatime) > 2:
        encoded_experiments.append((static_values, Deltatime, polarization, Uncertainty))
        # log_message(f"Creating Encoded Experiments (appending parameters, time array, polarization array and uncertainty array)") #Clutter log
    log_message(f"    load_experiments: Loaded {len(encoded_experiments)} experiments.")
    return encoded_experiments, static_df.columns.tolist()


#################################################################################################

def augment_experiments(original_experiments, num_augmentations=5, base_seed=42):
    """
    Arguments:
        original_experiments (list): A list, for all polarization decays like encoded_experiments in last function:
            original_experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...]
        num_augmentations (int): The number of augmentations done per experiment.
        base_seed (int): A seed that guarantees reproducibility
    
    Outputs:
        A list that concatenates the original experiments with the augmented ones.
        Instead of being a list of len(original_experiments) it becomes a list of
        len(original_experiments) * (1+ num_augmentations)
    
    Notes: Due to the stochastic nature of neutron detection, most measurements, if
    repeated under the same conditions, will yield different results. Of course, all
    measurements converge (with uncertainty) to what we can say is "the true value".
    Therefore, we can duplicate the experiments adding noise to the measurements to obtain
    "hypothetical" measurements that expand the data base. It was decided that the
    uncertainty won't be modified. To decide what type of noise could be applied two
    possibilities were considered. The first one was to suppose that sensor detection
    follows a Poisson distribution (law of rare events). The second one was to suppose
    that it follows a Normal Distribution of mean the measured value and width the uncertainty.
    As a poissonian distribution converges stochastically (in probability and distribution
    but not almost sure nor in L^p) to a normal distribution, it is safe to assume that they
    converge to the same result so the gaussian approach was selected. Also, the measurements
    are not raw counts but a function of them. P=(n^+ - n^-)/(n^+ + n^-)
    We have no direct data of the number of counts (the parameter of a Posissonian distribution)
    but a rough estimate points to them being (worst case scenario) within the order of the millions
    (After 100 counts, poisson distributions vary very little from normal distributions).
    Threfore, count detection can be approximated to a normal distribution and a linear
    combination of random variables distributed as normal distributions is also a random variable
    with a normal distribution. Therefore, it is safe to assume that sensor measurements can be
    modelled after a gaussian distribution.  
    """
    
    log_message(f"    augment_experiments: Augmenting {len(original_experiments)} a number of {num_augmentations} times")
    augmented_experiments = []
    for idx, (static, time, polar, uncertainty) in enumerate(original_experiments):
        for n in range(num_augmentations):
            seed = hash((idx, n, base_seed)) % 2**32
            rng = np.random.default_rng(seed)
            noise = rng.normal(loc=0.0, scale=uncertainty)
            new_polar = polar + noise
            if LogNoise:
                log_message(f"    augment_experiments:       Augmented experiment {idx} #{n} with seed {seed}")
            augmented_experiments.append((static, time, new_polar, uncertainty))
    log_message(f"    augment_experiments: Augmented to {len(original_experiments + augmented_experiments)} experiments")
    return original_experiments + augmented_experiments

#######################################################################################3

def build_dataset(experiments, mode="PolarizationD3"):
    """
    Arguments:
        experiments (list): It is a list like the output of load_experiments or augment_experiments
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...]
        mode (str): The column of polarization that will be used. It can be either 'SoftPolarizationD3' or 'PolarizationD3'.
    
    Output:
        1. Xs. An array of lists. Shape (number_of_samples, number_static_features) Each list contains all the original static parameters (hot encoded) plus the first and last polarization values with uncertainty.
           To be precise, they get added in this order: static parameters + initial time + initial polarization + final time + final polarization.
           For all samples, the static features get added to this list. This means that, if an experiment has M measurements, two get added to the parameter
           list and then those parameter features get written M-2 times in this array
        2. Xt. An array of time. Shape (number_of_samples, 1). All time values that are not used as parameters get added to this array. However they are not
           saved directly. If they are for example 0,120,250 then they get saved as [0],[120],[250]. The reason is compatibility with Keras/TensorFlow.
        3. y. An array of polarization. It is the same as Xt but with polarization values (the type of polarization is determined by _mode_)
        4. err. An array of polarization uncertainties. It is the same as Xt and y but with polarization uncertainties.
    Notes:
        If there are only two rows then the file gets skipped. It shouldn't happen but there is logic for it. The reason why it gets skipped is because
        there would not be any values left to use for training or validation

    
    """

    Xs, Xt, y, u = [], [], [], []
    log_message(f"    build_dataset: Starting build_dataset for column {mode} ")
    log_message(f"    build_dataset: Number of experiments to process: {len(experiments)} (should be (num_augmentations+1)*(number_of_experiments-1)")
    
    for exp_idx, (static_params, delta_time, polarization, uncertainty) in enumerate(experiments):
        if len(delta_time) < 2:
            log_message(f"    ****build_dataset:       Skipping experiment {exp_idx}: too few data points (len={len(delta_time)})")
            continue

        
        if LogNoise:
            log_message(f"    build_dataset: Adding First and Last Polarization (with time) values as static parameters") #Clutter log
        init_idx = 0
        final_idx = -1
        initial_dt, initial_p = delta_time[init_idx], polarization[init_idx]
        final_dt, final_p = delta_time[final_idx], polarization[final_idx]

        static_vector = static_params + [
            initial_dt, initial_p,
            final_dt, final_p
        ]
        if LogNoise:
            log_message(f"    build_dataset: Experiment {exp_idx}: static_vector length={len(static_vector)} (should be 10 (three parameters, CellID hot encoded creates three posibilities, four for the initial and final polarization) ") #Clutter log
        if LogNoise:
            log_message(f"    build_dataset: Building samples Static+time+polarization") #Clutter log
        

        for t, p, err in zip(delta_time[1:-1], polarization[1:-1], uncertainty[1:-1]): 
            Xs.append(static_vector)
            Xt.append([t])
            y.append(p)
            u.append(err) #We will ignore always uncertainty in parameters and even if they are not used, we will keep uncertainties in the data sets(same dimensions everywhere)

    log_message(f"    build_dataset: Number of experiments processed for mode '{mode}': {len(experiments)}")
    log_message(f"    build_dataset: Final dataset shapes: Xs: {np.array(Xs).shape}, Xt: {np.array(Xt).shape}, y: {np.array(y).reshape(-1, 1).shape}")
    return np.array(Xs), np.array(Xt), np.array(y).reshape(-1, 1), np.array(u).reshape(-1, 1)

#################################################################################

def nll_loss(y_true, y_pred):
    """
    Arguments:
        y_true (array): An array of the real values.
        y_pred (array): An array of the predicted values
    Output:
     A scalar tensorflow tensor ( () ) with the value of the loss as the the mean Gaussian negative log-likelihood over the batch
    Notes:
        This loss function is not just a pure Negative Loss Likelyhood (NLL).
        ML algorithms require a way to tell the algorithm if it is learning or not. The most standard practice is with a Loss function.
        If the loss value goes down that means that the algorithm is learning and, if a step increases the loss, then it is punished and
        tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative
        log-likelihood of a normal distribution 
        
        NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
        
        where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value.
        Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as
        $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

        However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough
        estimate on what the uncertainty of a set predictions looks like. Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector
        of $N$ predicted values. It can be considered as a random vector of variance:
        
        $Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
        
        where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of
        the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as
        $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model
        it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but
        using squared values punished big discrepancies in a stronger way).

        $StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
        
        where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not
        $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the
        loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental
        uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

        Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to
        lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated
        uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than
        underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model
        returns then is closer to a poorly calculated linear fit. Therefore, an addition penalty was added.

        $OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
        
        where $C$ is a constant used to control the weight of this penalty
    """

    mu = y_pred[:, 0:1]
    log_var = y_pred[:, 1:2]

    # Base Gaussian NLL
    precision = tf.exp(-log_var)
    nll = tf.reduce_mean(0.5 * (log_var + tf.square(y_true - mu) * precision))


    mu_centered = mu - tf.reduce_mean(mu)
    sigma_ref = tf.sqrt(tf.reduce_mean(tf.square(mu_centered)) + 1e-6)
    Stray_penalizer = 1e-2 #Parameter used to limit how far the uncertainty predictions stray from the experimental data 
    OverUncert_penalizer  = 5e-3 #Parameter used for punishing overestimated uncertainties
    log_var_prior = tf.math.log(sigma_ref ** 2) #Order of magnitude of what uncertainties should look like

    # Prior penalty. It penalizes if uncertainty strays too much from the experimental value
    #It avoids underestimation of uncertainty and overestimation
    penalty_stray = Stray_penalizer * tf.reduce_mean(tf.square(log_var - log_var_prior))

    # Asymmetric penalty: punish overestimation more than underestimation
    penalty_overestimate = OverUncert_penalizer * tf.reduce_mean(tf.nn.relu(log_var - log_var_prior))

    return nll + penalty_stray + penalty_overestimate

## 4. Model Architecture

It consists of a single function called _Define\_Complexity_. Given the name of a model it defines the model function of the ML algorithm. To be precise, it defines a function called _build_model_ every time _Define\_Complexity_ runs. If _Define\_Complexity_ gets run another time, it will define (possibly) another _build_model_ if the _Complexity_ variable changes

In [4]:
def Define_Complexity(Complexity):
    """
    Arguments: 
        Complexity (str): It is the name of the model
    Output: 
        None, but it defines the model architecture. It can be accessed outside of this function by calling build_model (once Definde_Complexity has ran)
    
    Characteristics of build_model:
        Arguments:
            input_dim_static (int): It is the number of static input features (the static parameters + 4)
            use_uncertainty (bool): It changes the output of the model to use (or not) uncertainty.
                The model will either train with mse and output just the mean or train wil NLL and output the mean and the log of the variance (σ^2)
        Output:
            A keras model.
        Notes: Here are the models used:
                Complex, Simple, Naif, Naif834, Naif838, Naif8316, Naif8332, Naif854, Naif858, Naif8516,
                Naif8532, Naif8104, Naif8108, Naif81016, Naif81032, Naif1634, Naif1638, Naif16316, Naif16332,
                Naif1654, Naif1658, Naif16516, Naif16532, Naif16104, Naif16108, Naif161016, Naif161032, Naif2434,
                Naif2438, Naif24316, Naif24332, Naif2454, Naif2458, Naif24516, Naif24532, Naif24104, Naif24108,
                Naif241016, Naif241032, NaifTwice1D883316, NaifTwice1D883332, NaifTwice1D883516, NaifTwice1D883532,
                NaifTwice1D885316, NaifTwice1D885332, NaifTwice1D885516, NaifTwice1D885532, NaifTwice1D843316,
                NaifTwice1D843332, NaifTwice1D843516, NaifTwice1D843532, NaifTwice1D845316, NaifTwice1D845332
                NaifTwice1D845516, NaifTwice1D845532, NaifTwice1D483316, NaifTwice1D483332, NaifTwice1D483516,
                NaifTwice1D483532, NaifTwice1D485316, NaifTwice1D485332, NaifTwice1D485516, NaifTwice1D485532,
                NaifTwice1D443316, NaifTwice1D443332, NaifTwice1D443516, NaifTwice1D443532, NaifTwice1D445316,
                NaifTwice1D445332, NaifTwice1D445516, NaifTwice1D445532, NaifTwiceDense414, NaifTwiceDense418
                NaifTwiceDense4116, NaifTwiceDense4124, NaifTwiceDense814, NaifTwiceDense818, NaifTwiceDense8116,
                NaifTwiceDense8124, NaifTwiceDense1614, NaifTwiceDense1618, NaifTwiceDense16116, NaifTwiceDense16124,
                NaifTwiceDense2414, NaifTwiceDense2418, NaifTwiceDense24116, NaifTwiceDense24124, NaifTwiceDense424,
                NaifTwiceDense428, NaifTwiceDense4216, NaifTwiceDense4224, NaifTwiceDense824, NaifTwiceDense828,
                NaifTwiceDense8216, NaifTwiceDense8224, NaifTwiceDense1624, NaifTwiceDense1628, NaifTwiceDense16216,
                NaifTwiceDense16224, NaifTwiceDense2424, NaifTwiceDense2428, NaifTwiceDense24216, NaifTwiceDense24224,
                SimpleNoDropout444, SimpleNoDropout448, SimpleNoDropout484, SimpleNoDropout488, SimpleNoDropout4164,
                SimpleNoDropout4168, SimpleNoDropout844, SimpleNoDropout848, SimpleNoDropout884, SimpleNoDropout888,
                SimpleNoDropout8164, SimpleNoDropout8168, SimpleNoDropout1644, SimpleNoDropout1648, SimpleNoDropout1684,
                SimpleNoDropout1688, SimpleNoDropout16164, SimpleNoDropout16168, Simple414414, Simple414418,
                Simple414424, Simple414428, Simple414814, Simple414818, Simple414824, Simple414828, Simple418414,
                Simple418418, Simple418424, Simple418428, Simple418814, Simple418818, Simple418824, Simple418828,
                Simple424414, Simple424418, Simple424424, Simple424428, Simple424814, Simple424818, Simple424824,
                Simple424828, Simple428414, Simple428418, Simple428424, Simple428428, Simple428814, Simple428818,
                Simple428824, Simple428828, Simple814414, Simple814418, Simple814424, Simple814428, Simple814814,
                Simple814818, Simple814824, Simple814828, Simple818414, Simple818418, Simple818424, Simple818428,
                Simple818814, Simple818818, Simple818824, Simple818828, Simple824414, Simple824418, Simple824424,
                Simple824428, Simple824814, Simple824818, Simple824824, Simple824828, Simple828414, Simple828418,
                Simple828424, Simple828428, Simple828814, Simple828818, Simple828824, Simple828828
    """
    log_message(f"Define_Complexity: Defining model {Complexity}")
    if Complexity == "Average":       
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh')(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Complex":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.3)(x_time)
            x_time = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = Dropout(0.3)(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.3)(x)
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-4))(x)
            x = Dropout(0.2)(x)
            x = Dense(16, activation='tanh')(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Simple":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.3)(x_time)
            x_time = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-2))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Naif":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif834":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif838":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif854":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif858":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif8532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif81016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif81032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        

 





        
    elif Complexity == "Naif1634":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif1638":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif1654":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif1658":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif16532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif161016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif161032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
 









    elif Complexity == "Naif2434":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif2438":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif2454":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif2458":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif24532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif241016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif241032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
        
        
        
        
        
        
    elif Complexity == "NaifTwice1D883316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D883332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D883516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D883532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D885316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D885332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D885516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D885532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D843316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D843332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D843516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D843532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D845316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D845332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D845516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D845532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    

        
        
        
        
    elif Complexity == "NaifTwice1D483316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D483332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D483516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D483532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D485316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D485332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D485516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D485532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D443316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D443332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D443516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D443532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D445316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D445332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D445516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D445532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
        
        
        
        
        
    elif Complexity == "NaifTwiceDense414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                    
    elif Complexity == "NaifTwiceDense4116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense4124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense1614":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense1618":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

        
        
        
        
    elif Complexity == "NaifTwiceDense424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                    
    elif Complexity == "NaifTwiceDense4216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense4224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense1624":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense1628":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
        
        
        
        
    elif Complexity == "SimpleNoDropout444":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout448":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
        
    elif Complexity == "SimpleNoDropout484":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout488":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout4164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout4168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          

    elif Complexity == "SimpleNoDropout844":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout848":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "SimpleNoDropout884":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout888":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout8164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout8168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model         
    elif Complexity == "SimpleNoDropout1644":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout1648":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "SimpleNoDropout1684":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout1688":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout16164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout16168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model         
        
        

        
        
        
    elif Complexity == "Simple414414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple414424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple414814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple414824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple418414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple418424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple418814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple418824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
    elif Complexity == "Simple424414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple424424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple424814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple424824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple428414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple428424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple428814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple428824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
        
        
        
        
        
    elif Complexity == "Simple814414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple814424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple814814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple814824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple818414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple818424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple818814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple818824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
    elif Complexity == "Simple824414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple824424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple824814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple824824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple828414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple828424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple828814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple828824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x_time)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    else:
        log_message(f"****Define_Complexity: UNKNOWN MODEL TYPE. FAIL INCOMING! Please check that the model name actually exists.")
    log_message(f"Define_Complexity: Model built\n")
    return build_model 

## 5. Model specific functions

Here we have functions that train, validate and fit the models. Some models require the variables to be scaled or will scale them. Extra precautions need to be taken into account

1. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    1.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation.
    
    1.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have only four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    1.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model and it trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for amorphous materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.


Also, five eight randomly picked models were tested with and without validation and with and without an asymetric uncertainty-overestimated penalizing loss. The result showed that the Loss update was better and validation did not increase performance (without validation, the results were slightly better).

2. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

3. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

    3.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
    3.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
    3.3. It builds the model depending on the use_uncertainty bool. (It changes the loss function and the output).
    
    3.4. It trains the model and returns its history (the trained model)
    
4. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
5. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the time points stored as static features are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still universal. However we can't fully say that it is a pure ML algorithm. The 'correctness' of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.

In [5]:
def model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose, callbacks=None):
    """
    Arguments: 
        model (keras model): This is the object that the build_model function returns.
        X_static_scaled (array): It is an array os size (total number of intermediate measured points (let's call them 'samples'), number of static features). 
                                 Using the variables form build_dataset, the number of static features is 4+len(static_parameters(hot encoded)).
                                 To be precise, it is just Xs scaled (an array of lists with the static features) but reshpaed into a 2d array
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values
        epochs (int): Number of training epochs (times the model tries to validate the data and recalculates its parameters)
        batch_size (int): The number of samples for every gradient update
        verbose (int): It limits how much training output is printed
        callbacks (str): It allows certain Keras callbacks (special code properties of keras). EarlyStopping, ReduceLROnPlateau or ModelCheckpoint are examples
    Outputs:
       A keras.callbacks.History object
    Note:
        It logs and runs model.fit() on a two-input Keras model and returns the training history.
        IT REQUIRES THE INPUTS (static, time and polarization) TO BE NORMALIZED/SCALED
        Training is not done using the uncertainties of the data.  
    """
    log_message(f"    Model_fitting: Training the model.")
    return model.fit(
        [X_static_scaled, X_time_scaled],
        y_scaled,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        callbacks=callbacks)

def model_prediction(model, X_static_scaled, X_time_scaled):
    """
    Arguments:
        X_static_scaled (array): A 2d array of Xs (samples, number of static features)
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values. It doesn't have to be the same as the one used in training
    Output:
        A 2d array with the predictions for those time values. It contains a column of the polarization predictions and another with the log of the variance
    Notes:
        IT REQUIRES THE INPUTS (static and time) TO BE NORMALIZED/SCALED
    """
    log_message(f"    model_prediction: Predicting {len(X_time_scaled)} time points")
    return model.predict([X_static_scaled, X_time_scaled], verbose=0)
        


def train(encoded_experiments, scaler_static, scaler_time, scaler_y, use_uncertainty, mode):
    """
    Arguments:
        encoded_experiments (list): A list like this:
            encoded_experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        scaler_static: It is a scikit-learn scaler for the static features. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_time: Analogously to scaler_static
        scaler_y: Analogously to scaler_static
        use_uncertainty (bool): It toggles whether it uses uncertainty for predictions or not. This changes the output of the model (mean and log_var or just the mean)
        mode: (str): It decides what polarization column to use. It can be 'SoftPolarizationD3' or 'PolarizationD3'
    Output:
        A fully trained model that can be used by model_predictions
    Notes:
        This function is the one responsible of scaling the inputs and training the model
        1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
        2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized.
           The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
        3. It builds the model depending on the use_uncertainty bool. (It changes the loss function and the output).
        4. It trains the model and returns its history (the trained model)
    
    """
    log_message(f"    train: Begin training and scaling with {len(encoded_experiments)} experiments.")
    X_static_all, X_time_all, y_all, u_all = build_dataset(encoded_experiments)


    log_message(f"    train: Scaling all data")
    X_static_scaled = scaler_static.transform(X_static_all) #It only fits to the scaler. IT DOESN'T OVERWRITE THEM. WHAT A WASTE OF 20 DAYS OF MY LIFE >:(
    X_time_scaled   = scaler_time.transform(X_time_all)
    y_scaled        = scaler_y.transform(y_all)

    

    model = build_model(X_static_all.shape[1], use_uncertainty=use_uncertainty)
    epochs = 300
    batch_size = 32
    log_message(f"    train: Training final model with epochs={epochs}, batch_size={batch_size} and use_uncertainty = {use_uncertainty}")
    model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose=0)
    return model

############################################################

def align_static_vectors(experiments, static_columns_training, static_columns_isolated):
    """
    Arguments:
        experiments (list): A list like this:
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        static_columns_training (list): A list of strings of all the static feature names used for training
        static_columns_isolated (list): A list of strings of all the static feature names of the isolated experiment
    Output:
        aligned_experiments is a list of experiments with static vectors aligned to the training feature order.
        To be precise (aligned_static, delta_time, polarization, uncertainty) where on the parameters where static_columns_isolated
        had no values get turned into zeros. So, if static_columns_isolated doesn´t have a parameter 'Parameter 6', then, in the
        original experiments lists, all numbers associated to 'Parameter 6' get turned to zero

    Notes:
        It helps the ML algorithm to focus on the important parameters
    """
    
    aligned_experiments = []
    for static, delta_time, polarization, uncertainty in experiments:
        static_dict = dict(zip(static_columns_isolated, static))
        aligned_static = [static_dict.get(col, 0.0) for col in static_columns_training]
        aligned_experiments.append((aligned_static, delta_time, polarization, uncertainty))
    log_message(f"    align_static_vectors: Vectors aligned")
    return aligned_experiments

#############################################################

def model_predict_sloped(model, X_static_scaled, X_time_scaled, m, n, scaler_y, scaler_time):
    """
    Arguments:
        model: A trained model
        X_static_scaled (array). A 2d array of Xs (samples, number of static features)
        X_time_scaled (array). A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        m (float): The slope used for correction
        n (float): The polarization shift for correction
        scaler_time: It is a scikit-learn scaler for the time arrays. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_y: Analogously to scaler_static
    Outputs
        y_pred_corrected_scaled is a 2D array where each column is the corrected predicted values (scaled) and the second one is the log of the variance (unchanged, a.k.a scaled)
    Notes:
        1. It predicts the polarization values
        2. It inverse transforms the predicted values and the time points (not the uncertainty)
        3. It calculates the correction curve in real units and corrects the predicted values.
        4. Finally, it scales back the corrected polarization values and joins them with the unchanged logarithm of the variance
        In summary, it substracts a linear function to the predicted values. If done correctly this makes it so that the polarization
        predictions at the time points stored as static features are the same as the measured polarization values at those times.
        This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still
        universal. However we can't fully say that it is a pure ML algorithm.
    """
    
    log_message(f"    model_predict_sloped: Correcting {len(X_time_scaled)} points by subtracting P(t) = {float(np.squeeze(m)):.3e} * t + {float(np.squeeze(n)):.3e}")
    y_pred_scaled = model_prediction(model, X_static_scaled, X_time_scaled)
    mean_scaled = y_pred_scaled[:, 0:1]  # shape (N,1)
    mean_real = scaler_y.inverse_transform(mean_scaled)  # shape (N,1)
    time_real = scaler_time.inverse_transform(X_time_scaled)  # shape (N,1)
    
    correction = m * time_real + n  # shape (N,1)
    mean_corrected_real = mean_real - correction  # shape (N,1)
    
    mean_corrected_scaled = scaler_y.transform(mean_corrected_real)  # shape (N,1)
    log_var_scaled = y_pred_scaled[:, 1:2]  # shape (N,1)
    y_pred_corrected_scaled = np.hstack([mean_corrected_scaled, log_var_scaled])  # shape (N,2)
    
    return y_pred_corrected_scaled

## 6. Isolation logic
To avoid leaks in the ML code a special pipeline was designed. First, an experiment gets removed form the training data base and a model is trained on the remaining experiments. Then, predicitions for the isolated experiment can be produced and their accuracy computed afterwards. The isolated experiment returns to the data base and a new one gets removed. If this process is looped for all experiments, we have a "Leave-One-Out" process to quantify how good the model predicts unseen experiments without sacrificing too much the size of the data base. This code is responsible for this process

In [6]:
def isolate_experiments(data_dir, isolated_dir, output_folder, use_uncertainty, num_augmentations, Correction, build_model):
    """
    Arguments:
        data_dir (str): The directory folder that contains all the files for the data base
        isolated_dir (str): The directory folder where the isolated experiment will be sent
        use_uncertainty (bool): A bool variable that can toggle whether the model uses or not uncertainties in their output (not during training)
        num_augmentations (int): The number of augmentations used in the model.
        Correction (bool): A bool that toggles between predictions with the slope correction or raw predictions
        build_model (function): It uses the model defined from Define_Complexity. That function gets called outside this function but the build_model 
        function needs to be given as an argument for isolate_experiments
    
    Outputs:
    
    Notes:
        1. It creates the folder where the isolated files will temporally reside
        2. It loops for all pairs of array and parameter files
        3. It then moves both, parameter and array, files to the isolated directory
        4. A subfolder on the main folder gets created with the name of the isolated experiment. All results will
           be stored here for this isolation iteration.
        5. Loop for all polarization modes to be processed. It can be SoftPolarizationD3, PolarizationD3 or both
        6. Load the rest of the files and augment them
        7. Obtain the time and polarization scalers. They are obtained using all time values and all polarization
           values including the soon-to-become static features
        8. Obtain the static features scaler.
        9. Save scalers locally
        10. Train the model. The data gets scaled during this step.
        11. Load the isolated experiment and align both sets of data (use align_static_vectors)
        12. Build manually the dataset for the isolated experiment
        13. Extract the initial and final points, add them as parameters and scale the entire static vector with the general scaler.
        14. Scale time and polarization arrays (only intermediate points). This is compatible with the scaling done to the rest of the experiments.
        15. Combine all structures so that the shape is correct for prediction.
        16. Check for shapes and create the final NumPy arrays of the experiment
        17. Extract the initial and final polarizations and keep a scaled and unscaled version of all. Then predict at those time points. 
        18. Prepare the linear correction and predict in the time points where we have data.
        The unscaled (physical units) outputs are pred_polar and pred_sigma.
        19. Plot the raw values (black), the predictions for those time points (blue) and predictions every 1000 seconds (red if there is no correction, green if there is correction)
        20. Save data
        21. Move all experiments back to the original folder


    
    Scaling is done using also the polarization and time measurements that become static features. This was done so
    that the ML algorithm can easily "figure out" that those parameters are related to polarizations and time moments.
    Another equally valid approach is to only scale the intermediate points and then scale the parameters independently.
    The downside of this first approach is that those static features get scaled twice (once as regular polarizations and
    time values and a second time between all static features). In practice both methods showed close to no difference and
    the second approach needs to juggle three scalers at once when working with the static vector. This proved to be difficult
    and, again, showed no real improvement.
    This scalers are obtained using MinMaxScaler. It finds the maximum and minimum of all values and normalizes all
    of them uniformly (if X is a value on the array, then it gets replaced by (X-min)/(max-min))
    Here is a list of all variables used
    
    OG for original scale, SC for scaled
        all_time: OG. All time values
        all_y: OG. All polarization values
        scaler_time: Scaler of ALL time
        scaler_y: Scaler of ALL polarizations
        scaler_static: Scaler of ALL parameters (WITH t_ini, t_fin, P_ini, P_fin)

        X_static_raw. OG. All static features
        X_time_raw. OG. All time values without initial and final
        y_raw. OG. All polarization values without initial and final
        u. OG. All uncertainties
        model. SC. 
        ____________________________________________________________________-
        Measured values:

        initial_dt. OG. t_ini
        initial_p. OG. P_ini
        final_dt. OG. t_fin
        final_p. OG. P_fin

        static. OG. Intermediate Isolated parameters 
        static_vector. OG. Isolated parameters (WITH t_ini, t_fin, P_ini, P_fin)
        static_scaled. SC. Isolated parameters (WITH t_ini, t_fin, P_ini, P_fin)

        time. OG. All isolated time values
        time_intermediate. OG. Intermediate Isolated time values 
        time_scaled. SC. Intermediate Isolated time values 

        pol. OG. All isolated polarizations
        pol_intermediate. OG. Intermediate Isolated polarization 
        pol_scaled. SC. Intermediate Isolated polarization 

        unc. OG. All Isolated uncertainties
        unc_intermediate: OG. Intermediate Isolated uncertainties
        ____________________________________________________________________
        Scaling and slope:
        
        scaled_isolated_experiments: SC. Intermediate full list
        X_static. SC. Isolated parameters (WITH t_ini, t_fin, P_ini, P_fin)
        X_time: SC. Intermediate Isolated time values
        y_true: SC. Intermediate Isolated polarization
        u: OG. Intermediate Isolated uncertainties     

        X_static_raw. OG. Isolated parameters (WITH t_ini, t_fin, P_ini, P_fin)           
        t0_raw == initial_dt
        p0_raw == initial_p
        tT_raw == final_dt  
        pT_raw == final_p

        t0_scaled_correct. SC. t_ini scaled as time
        tT_scaled_correct. SC. t_fin scaled as time
        p0_scaled_correct. SC. P_ini scaled as polarizations
        pT_scaled_correct. SC. P_fin scaled as polarizations

        pred_initial_scaled. SC. P_ini predicted
        pred_final_scaled. SC. P_fin predicted
        p0_pred. OG. P_ini predicted
        pT_pred. OG. P_fin predicted

        m_raw. OG. Slope in real units
        n_raw. OG. Origin in the Polarization axis
        y_pred_raw. SC. Corrected polarization predicitons including extremes
        mean_pred. OG. Corrected polarization predicitons including extremes
        pred_polar == mean_pred
        log_var_scaled. SC. Log variance predictions including extremes
        var_scaled. SC. Variance predictions including extremes 
        scale_y. OG. Data-wise polarization range
        var_rescaled. OG. Variance predictions including extremes.
        pred_sigma. OG. Uncertainty predicitons including extremes.
        _______________________________________________________________________

        Iso_Interm_Polar_SC: SC. Intermediate Isolated polarization
        Iso_Interm_Polar_OG: OG. Intermediate Isolated polarization
        Iso_Interm_Uncert_OG: OG. Intermediate Isolated uncertainties
        Iso_Interm_Time_SC: SC. Intermediate Isolated time
        Iso_Interm_Time_OG: OG. Intermediate Isolated time

        _______________________________________________________________________
        
        Grid_time_OG: OG. Smooth time array 
        Grid_time_SC: SC. Smooth time array

        Grid_polar_Logvar_Sloped_SC: SC. Predicted smooth polarization with logvar array and slope correction
        Grid_polar_Logvar_Sloped_OG: OG. Predicted smooth polarization with logvar array and slope correction
        Grid_Logvar_Sloped_SC: SC. Predicted smooth logvar array and slope correction
        Grid_Var_Sloped_SC: SC. Predicted smooth variance array and slope correction
        Grid_Var_Sloped_OG: OG. Predicted smooth variance array and slope correction
        Grid_Uncert_Sloped_OG: OG. Predicted smooth uncertainty array and slope correction
        Grid_polar_Sloped_OG: OG. Predicted smooth polarization array and slope correction

        Grid_polar_Logvar_SC: SC. Predicted smooth polarization with logvar array 
        Grid_polar_Logvar_OG: OG. Predicted smooth polarization with logvar array 
        Grid_Logvar_SC: SC. Predicted smooth logvar array
        Grid_Var_SC: SC. Predicted smooth variance array
        Grid_Var_OG: OG. Predicted smooth variance array
        Grid_Uncert_OG: OG. Predicted smooth uncertainty array
        Grid_polar_OG: OG. Predicted smooth polarization array
    
    """
    # 1. Prepare the folder where the isolated experiments will reside
    log_message(f"IE: Create the isolated folder ")
    data_dir = Path(data_dir).resolve()
    isolated_dir = Path(isolated_dir).resolve()
    output_folder = Path(output_folder).resolve()

    # Get all *_Parameters.txt files to derive base names
    log_message(f"IE: Obtaining all the names of the experiments")
    param_files = list(data_dir.glob("*_Parameters.txt"))
    base_names = [f.stem.replace("_Parameters", "") for f in param_files]
    
    #2. Loop over all pairs of data files. Both the parameter one and the arrays one are needed
    for base_name in base_names: 
        log_message(f"\n\nIE: Begin isolation of {base_name}")

        file_param = data_dir / f"{base_name}_Parameters.txt"
        file_data = data_dir / f"{base_name}.txt"

        if not (file_param.exists() and file_data.exists()):
            log_message(f"****IE:      Skipping {base_name}: One of the required files does not exist.")
            continue

        #3. Move both files to isolated folder 
        shutil.move(long_path(file_param), long_path(isolated_dir / file_param.name))
        shutil.move(long_path(file_data), long_path(isolated_dir / file_data.name))
        log_message(f"IE: Successfully sent {base_name} to MLAmorphousIsolatedExperiment")

        #4. Create the subfolder where all information regarding the model trained without experiment {base_name} will reside
        output_subfolder = output_folder / f"Missing{base_name}"
        output_subfolder.mkdir(parents=True, exist_ok=True)
        log_message(f"IE: Created folder: {output_subfolder}")


        #5. Train and predict using both modes (PolarizationD3" and "SoftPolarizationD3"). If only one is selected change the logic with: for mode in ["PolarizationD3", "SoftPolarizationD3"]: 
        for mode in ["PolarizationD3"]: 
            log_message(f"IE: Processing mode: {mode}")

            #6. Load all experiments and augment them. These experiments will only be used for training the model and creating the scalers
            log_message(f"IE: Load all other experiments and augment them {num_augmentations}")
            experiments, static_columns = load_experiments(data_dir, polarization_column=mode)
            encoded_experiments = augment_experiments(experiments, num_augmentations, base_seed=42)
            
            #7. Scalers. All polarization and time values are scaled using all the experiments (except the isolated one). The reason for excluding them is to avoid information
            #leacks. The static parameters are scaled afterwards but they are also scaled with all but one experiments
            log_message(f"IE: Scaling all training experiments...")
            all_time = []
            all_y = []
            
            for static, time, pol, _ in encoded_experiments:
                all_time.append(time)
                all_y.append(pol)
            
            all_time = np.concatenate(all_time).reshape(-1, 1) #unscaled
            all_y = np.concatenate(all_y).reshape(-1, 1) #unscaled
            
            scaler_time = MinMaxScaler().fit(all_time) #fit to all
            scaler_y = MinMaxScaler().fit(all_y) #fit to all
            
            #8. Prepare all the experiments so that they can be introduced in the training functions
            X_static_raw, X_time_raw, y_raw, u = build_dataset(encoded_experiments, mode=mode)
            scaler_static = MinMaxScaler().fit(X_static_raw) #fit to all

            if LogNoise:
                log_message(f"IE:      Scaler static min_: , {scaler_static.min_}")
                log_message(f"IE:      Scaler static scale_: , {scaler_static.scale_}")
                log_message(f"IE:      Scaler static data_min_: , {scaler_static.data_min_}")
                log_message(f"IE:      Scaler static data_max_: , {scaler_static.data_max_}")
                
                log_message(f"IE:      Scaler time min_: , {scaler_time.min_}")
                log_message(f"IE:      Scaler time scale_: , {scaler_time.scale_}")
                log_message(f"IE:      Scaler time data_min_: , {scaler_time.data_min_}")
                log_message(f"IE:      Scaler time data_max_: , {scaler_time.data_max_}")
                
                log_message(f"IE:      Scaler polarization min_: , {scaler_y.min_}")
                log_message(f"IE:      Scaler polarization scale_: , {scaler_y.scale_}")
                log_message(f"IE:      Scaler polarization data_min_: , {scaler_y.data_min_}")
                log_message(f"IE:      Scaler polarization data_max_: , {scaler_y.data_max_}")
            
            #9. Save the scalers. This is useful because otherwise we have no way to use the model outside of the loop (because we don't know how to unscale the values)
            joblib.dump(scaler_static, long_path(output_subfolder / f"scaler_static_{mode}.pkl"))
            joblib.dump(scaler_time, long_path(output_subfolder / f"scaler_time_{mode}.pkl"))
            joblib.dump(scaler_y, long_path(output_subfolder / f"scaler_y_{mode}.pkl")) 
            log_message(f"IE: Saved scalers for mode {mode} to: {output_subfolder}")

            #10. Train and save the model on the augmented experiments. The experiments enter unscaled and get scaled inside)
            log_message(f"IE: Train the model")
            model = train(
                encoded_experiments=encoded_experiments,  
                scaler_static=scaler_static,
                scaler_time=scaler_time,
                scaler_y=scaler_y,
                use_uncertainty=use_uncertainty,
                mode=mode)

            model_path = output_subfolder / f"model_{mode}.keras"
            model.save(long_path(model_path))
            log_message(f"IE: Saved model to: {model_path}")
            
            #11. Load isolated experiment. Predictions will be done on this experiment. Aligning is required
            log_message(f"IE: Load isolated experiment: {base_name}")
            isolated_experiments, static_columns_isolated = load_experiments(isolated_dir, polarization_column=mode)
            isolated_experiments_aligned = align_static_vectors(
                isolated_experiments,
                static_columns_training=static_columns,
                static_columns_isolated=static_columns_isolated) #all unscaled
            if LogNoise:
                log_message(f"IE:      Number of isolated experiments aligned (should be one): {len(isolated_experiments_aligned)}")
                log_message(f"IE:      Example static vector (pre-scale): {isolated_experiments_aligned[0][0]}")
                log_message(f"IE:      Example time vector (pre-scale): {isolated_experiments_aligned[0][1]}")
                log_message(f"IE:      Example polarization vector (pre-scale): {isolated_experiments_aligned[0][2]}")


            #12. Building manually the dataset for the isolated experiment
            log_message(f"IE: Scaling static vector for isolated experiment. Formatting the isolated experiment.")
            for static, time, pol, unc in isolated_experiments_aligned: #all unscaled
                if len(time) < 2:
                    log_message(f"****IE: Skipping experiment {i} due to insufficient time points")
                    continue  # skip too-short experiments
            
                #13 Extract the initial and final points, add them as parameters and scale the entire static vector with the general scaler
                initial_dt, initial_p = time[0], pol[0]
                final_dt, final_p = time[-1], pol[-1]
                static_vector = static + [initial_dt, initial_p, final_dt, final_p] #unscaled
                if LogNoise:
                    log_message(f"IE: Experiment static vector length (pre-scale): {len(static_vector)}")
                    log_message(f"IE: Experiment static vector (pre-scale): {static_vector}")
                static_scaled = scaler_static.transform(np.array(static_vector).reshape(1, -1)).flatten() #scaled
                if LogNoise:
                    log_message(f"IE: Experiment static vector (scaled): {static_scaled}")
                
                #14 Scale time and polarization arrays (only intermediate points). This is compatible with the scaling done to the rest of the experiments
                time_intermediate = np.array(time[1:-1]).reshape(-1, 1) # unscaled
                time_scaled = scaler_time.transform(time_intermediate).flatten() #scaled
                if LogNoise:
                    log_message(f"IE: Experiment time (pre-scale): {time_intermediate.flatten()}")
                    log_message(f"IE: Experiment time (scaled): {time_scaled}")
                pol_intermediate = np.array(pol[1:-1]).reshape(-1, 1) #unscaled
                pol_scaled = scaler_y.transform(pol_intermediate).flatten() #scaled
                if LogNoise:
                    log_message(f"IE: Experiment polarization (pre-scale): {pol_intermediate.flatten()}")
                    log_message(f"IE: Experiment polarization (scaled): {pol_scaled}")
                unc_intermediate = np.array(unc[1:-1]).reshape(-1, 1) #unscaled
                if LogNoise:
                    log_message(f"IE: Experiment uncertainty: {unc_intermediate.flatten()}")

                
                #15. Combine all structures so that the shape is correct for prediction
                scaled_isolated_experiments = []
                scaled_isolated_experiments.append((
                    static_scaled, time_scaled, pol_scaled, unc_intermediate.flatten()
                )) #scaled
                
                
                #16 Check for shapes and create the final NumPy arrays of the experiment
                X_static, X_time, y_true, u = [], [], [], []

                for static_scaled, time_scaled, pol_scaled, unc_intermediate in scaled_isolated_experiments:
                    for t, p, err in zip(time_scaled, pol_scaled, unc_intermediate):
                        X_static.append(static_scaled)   # already includes appended time/polarization info
                        X_time.append([t])               # wrap to preserve 2D structure for Conv1D
                        y_true.append(p)
                        u.append(err)
                X_static = np.array(X_static) #scaled
                X_time = np.array(X_time) #scaled
                y_true = np.array(y_true).reshape(-1, 1) #scaled
                u = np.array(u).reshape(-1, 1) #unscaled

                if LogNoise:
                    log_message(f"  X_static.shape: {X_static.shape}")
                    log_message(f"  X_time.shape: {X_time.shape}")
                    log_message(f"  y_true.shape: {y_true.shape}")
                    log_message(f"  X_static[0]: {X_static[0]} (scaled)")

            #17 Extract the initial and final polarizations and keep a scaled and unscaled version of all. Then predict at those time points. 
            #Unscaling needs to be done with static_scaler but then we need to scale them as polarizations or as time points with their respective scalers so that we can use model_predict
            log_message(f"IE:   Use slope correction.")
            X_static_raw = scaler_static.inverse_transform(X_static)
            # Extract static input (already scaled)
            x_static_single = X_static[0:1]  # shape (1, D_static)
            t0_raw = X_static_raw[0, -4].reshape(1, -1) #We don't use initial_dt, final_dt,... to guarantee that they are model-like values to help the model identify them clearer
            p0_raw = X_static_raw[0, -3].reshape(1, -1)
            tT_raw = X_static_raw[0, -2].reshape(1, -1)
            pT_raw = X_static_raw[0, -1].reshape(1, -1)

            t0_scaled_correct = scaler_time.transform(t0_raw)
            tT_scaled_correct = scaler_time.transform(tT_raw)
            p0_scaled_correct = scaler_y.transform(p0_raw)
            pT_scaled_correct = scaler_y.transform(pT_raw)
            
            # Predict scaled polarizations at those time points
            pred_initial_scaled = model_prediction(model, x_static_single, t0_scaled_correct)
            pred_final_scaled   = model_prediction(model, x_static_single, tT_scaled_correct)
            
            # Inverse transform predictions to physical (real) polarizations
            p0_pred = scaler_y.inverse_transform(pred_initial_scaled[:, 0:1])
            pT_pred = scaler_y.inverse_transform(pred_final_scaled[:, 0:1])

            if LogNoise:
                log_message(f"t0 (real) = {t0_raw}, tT (real) = {tT_raw}")
                log_message(f"p0_pred (real) = {p0_pred}, pT_pred (real) = {pT_pred}")
                log_message(f"p0 (true) = {p0_raw}, pT (true) = {pT_raw}")


            #18 Prepare the linear correction and predict in the time points where we have data
            m_raw = ( (p0_pred-p0_raw) - (pT_pred-pT_raw) ) / (t0_raw-tT_raw)
            n_raw = (p0_pred-p0_raw) - m_raw * t0_raw
            if Correction == True:
                log_message(f"IE:   Fixing initial and final points by substracting P(t) = {float(np.squeeze(m_raw)):.3e} * t + {float(np.squeeze(n_raw)):.3e}")
                y_pred_raw = model_predict_sloped(model, X_static, X_time, m_raw, n_raw, scaler_y, scaler_time) #Input es scaled, parametros unsacles, output es scaled también
            else:
                y_pred_raw = model_prediction(model, X_static, X_time)
                
            #To obtain the results we separate the value predictions form the log variance. The results need to be unscaled to real units while the uncertainty needs to be 
            #exponenciated (to obtain a variance), multiplied by the same factor that is used in MinMaxScaler and finally convert to uncertainty (take the square root)
            mean_pred = scaler_y.inverse_transform(y_pred_raw[:, 0:1]).flatten() #unscaled
            log_var_scaled = y_pred_raw[:, 1]  
            var_scaled = np.exp(log_var_scaled)
            scale_y = scaler_y.data_max_[0] - scaler_y.data_min_[0]  # scale factor from MinMaxScaler 
            # Variance rescales by the square of the scale factor
            var_rescaled = var_scaled * (scale_y ** 2) #unscaled
            pred_sigma = np.sqrt(var_rescaled) #unscaled
            pred_polar = mean_pred #unscaled

            #19. Plot the raw values (black), the predictions for those time points (blue) and predictions every 1000 seconds (red if there is no correction, green if there is correction)
            if use_uncertainty:
                suffix="Uncertainty"
            else:
                suffix="noUncertainty"
            #Begin plotting:
            color_true = "black"
            color_pred = "blue"
            color_grid = "green"
            color_raw = "red"
            log_message(f"IE:   Plot raw values, predictions for those time points and predictions every 1000 seconds.")
            # Flatten inputs
            Iso_Interm_Polar_SC = y_true.flatten() #scaled
            Iso_Interm_Uncert_OG = u.flatten() #unscaled
            Iso_Interm_Time_SC = X_time.flatten()  #scaled
            # Reshape to (-1, 1) for scaler inverse transform
            Iso_Interm_Polar_SC_reshaped = Iso_Interm_Polar_SC.reshape(-1, 1)
            Iso_Interm_Time_SC_reshaped = Iso_Interm_Time_SC.reshape(-1, 1)
            
            # Inverse transform to get unscaled values
            Iso_Interm_Polar_OG = scaler_y.inverse_transform(Iso_Interm_Polar_SC_reshaped).flatten()
            Iso_Interm_Time_OG = scaler_time.inverse_transform(Iso_Interm_Time_SC_reshaped).flatten()

            # Define grid for red line

            Grid_time_OG = np.arange(0, max(Iso_Interm_Time_OG) + 1000, 1000).reshape(-1, 1)
            
            static_scaled = np.tile(X_static[0], (len(Grid_time_OG), 1)) #SCALED

            Grid_time_SC = scaler_time.transform(Grid_time_OG)

            # Predict on grid
            if Correction == True:
                Grid_polar_Logvar_Sloped_SC = model_predict_sloped(model, static_scaled, Grid_time_SC, m_raw, n_raw, scaler_y, scaler_time) #scaled
                Grid_polar_Sloped_OG = scaler_y.inverse_transform(Grid_polar_Logvar_Sloped_SC[:, 0:1]) #unscaled
                Grid_Logvar_Sloped_SC = Grid_polar_Logvar_Sloped_SC[:, 1:2]  # Don't transform this
                Grid_Var_Sloped_SC = np.exp(Grid_Logvar_Sloped_SC)
                # Variance rescales by the square of the scale factor
                Grid_Var_Sloped_OG = Grid_Var_Sloped_SC * (scale_y ** 2)
                Grid_Uncert_Sloped_OG = np.sqrt(Grid_Var_Sloped_OG) #unscaled    

            Grid_polar_Logvar_SC = model_prediction(model, static_scaled, Grid_time_SC) #scaled
            Grid_polar_OG = scaler_y.inverse_transform(Grid_polar_Logvar_SC[:, 0:1]) #unscaled
            Grid_Logvar_SC = Grid_polar_Logvar_SC[:, 1:2]  # Don't transform this
            Grid_Var_SC = np.exp(Grid_Logvar_SC)
            # Variance rescales by the square of the scale factor
            Grid_Var_OG = Grid_Var_SC * (scale_y ** 2)
            Grid_Uncert_OG = np.sqrt(Grid_Var_OG) #unscaled   
                
           
            fig, ax = plt.subplots(figsize=(8, 5))
            # base_name = "PolarizationD3_CaFeAl_1_5_6_24_0_MillerIndex_(0,0,2)"
            
            # Split by underscores
            parts = base_name.split("_")
            
            # Extract the fixed portion you want: "CaFeAl_1_5_6_24_0"
            core_name = "_".join(parts[1:6])  
            
            # Build new title
            title = f"{core_name}_{Complexity}_{num_augmentations}"
            ax.set_title(title)
            ax.grid(True)
            
            # Black dots with error bars (truth). Using real units
            ax.scatter(Iso_Interm_Time_OG, Iso_Interm_Polar_OG, color=color_true, marker='o', s=40, label="Measurements")
            ax.errorbar(Iso_Interm_Time_OG, Iso_Interm_Polar_OG, yerr=Iso_Interm_Uncert_OG, fmt='none',
                        ecolor=color_true, alpha=0.6, capsize=3, label="Measured ±σ")
            
            # Blue dots with model error. should already be in real units
            ax.scatter(Iso_Interm_Time_OG, mean_pred, color=color_pred, marker='x', s=40, label="Prediction")
            ax.errorbar(
                Iso_Interm_Time_OG,
                mean_pred,
                yerr=pred_sigma,
                fmt='none',
                ecolor=color_pred,
                alpha=0.6,
                capsize=3,
                label="Prediction ±σ"
            )
            
            # Green line + band for prediction grid
            ax.scatter(Grid_time_OG.flatten(), Grid_polar_Sloped_OG.flatten(), color=color_grid, alpha=0.7, label="Smooth Prediction", marker='x', s=40)
            ax.fill_between(
                Grid_time_OG.flatten(),
                (Grid_polar_Sloped_OG - Grid_Uncert_Sloped_OG).flatten(),
                (Grid_polar_Sloped_OG + Grid_Uncert_Sloped_OG).flatten(),
                color=color_grid,
                alpha=0.2,
                label="Smooth Prediction ±1σ")
            
            #Red for no corrections
            ax.scatter(Grid_time_OG.flatten(), Grid_polar_OG.flatten(), color=color_raw, alpha=0.7, label="Smooth no Correction", marker='x', s=40)
            ax.fill_between(
                Grid_time_OG.flatten(),
                (Grid_polar_OG - Grid_Uncert_OG).flatten(),
                (Grid_polar_OG + Grid_Uncert_OG).flatten(),
                color='red',
                alpha=0.1,
                label="Smooth no Correction ±1σ")
            
            
            y_min = np.min(Iso_Interm_Polar_OG - Iso_Interm_Uncert_OG) - 0.02
            y_max = np.max(Iso_Interm_Polar_OG + Iso_Interm_Uncert_OG) + 0.02
            ax.set_ylim([y_min, y_max])
            
            ax.legend()
            fig.tight_layout()
            
            # Save Figure
            fig_path = output_subfolder / f"Missing_{base_name}.jpg"
            save_path = long_path(str(fig_path))
            fig.savefig(save_path, dpi=100)
            plt.close(fig)
            del fig, ax
            gc.collect()
            log_message(f"IE:   Saved figure to: {fig_path}")
            
            # Plot difference
            delta_polar = mean_pred = mean_pred - Iso_Interm_Polar_OG
            delta_uncertainty = np.sqrt(pred_sigma**2 + Iso_Interm_Uncert_OG**2)
            
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.set_title(f"Difference_{base_name}")
            ax.grid(True)
            ax.scatter(Iso_Interm_Time_OG, delta_polar, color="blue", marker="o", s=40, label="Predicted - True")
            ax.errorbar(Iso_Interm_Time_OG, delta_polar, yerr=delta_uncertainty, fmt='none', ecolor="blue", alpha=0.6, capsize=3, label="±σ")
            ax.axhline(0, color='gray', linestyle='--', linewidth=1)
            ax.set_xlabel("DeltaTime")
            ax.set_ylabel("Predicted - True Polarization")
            ax.legend()
            fig.tight_layout()
            
            fig_path = output_subfolder / f"Difference_{base_name}.jpg"
            save_path = long_path(str(fig_path))
            fig.savefig(save_path, dpi=100)
            plt.close(fig)
            log_message(f"IE:   Saved difference plot to: {fig_path}")
            
            #20. Save data
            diff_data_path = output_subfolder / f"Difference_{base_name}.txt"
            with open(long_path(diff_data_path), 'w') as f:
                f.write("DeltaTime\tDeltaPolarization\tDeltaUncertainty\n")
                for t, dp, du in zip(Iso_Interm_Time_OG, delta_polar, delta_uncertainty):
                    f.write(f"{t:.6f}\t{dp:.6f}\t{du:.6f}\n")
            log_message(f"IE:   Saved difference data to: {diff_data_path}")

            raw_data_path = output_subfolder / f"RawData_{mode}_Missing{base_name}.txt"
            with open(long_path(raw_data_path), 'w') as f:
                f.write("DeltaTime\tRealPolarizationD3\tErrRealPolarizationD3\n")
                for t, p, e in zip(time_intermediate, pol_intermediate, unc_intermediate):
                    f.write(f"{t.item():.6f}\t{p.item():.6f}\t{e.item():.6f}\n")
            log_message(f"IE:   Saved raw data to: {raw_data_path}")

            predicted_data_path = output_subfolder / f"PredictedData_{mode}_Missing{base_name}.txt"
            with open(long_path(predicted_data_path), 'w') as f:
                f.write("GridTime\tPredictedPolarizationD3\tErrPredictedPolarizationD3\n")
                for t, p, s in zip(Grid_time_OG.flatten(), Grid_polar_Sloped_OG.flatten(), Grid_Uncert_Sloped_OG.flatten()):
                    f.write(f"{t:.6f}\t{p:.6f}\t{s:.6f}\n")
            log_message(f"IE:   Saved predicted data to: {predicted_data_path}")

            predicted_points_path = output_subfolder / f"PredictedPoints_{mode}_Missing{base_name}.txt"
            with open(long_path(predicted_points_path), 'w') as f:
                f.write("DeltaTime\tPredictedPolarizationD3\tErrPredictedPolarizationD3\n")
                for t, p, s in zip(Iso_Interm_Time_OG, pred_polar, pred_sigma):
                    f.write(f"{t:.6f}\t{p:.6f}\t{s:.6f}\n")
            log_message(f"IE:   Saved predicted point data to: {predicted_points_path}")
            
            #All this is to clean up files to avoid memory fills
            
            process = psutil.Process(os.getpid())
            tf.keras.backend.clear_session()
            del model
            gc.collect()
            #def mem():
            #    return process.memory_info().rss / 1e9
            #print(f"Memory after cleanup: {mem():.2f} GB") 

        #21. Move all experiments back to the original folder
        shutil.move(long_path(isolated_dir / os.path.basename(file_param)), long_path(file_param))
        shutil.move(long_path(isolated_dir / os.path.basename(file_data)), long_path(file_data))
        log_message(f"IE: Returned {base_name} to MLDataBase")




## 7 Main Code

Here we just have a code cell that uses isolate_experiments to do the Leave-One-Out process. It creates the folders and loops over all model names and number fo augmentations. For each combination, it runs the _isolated\_experiments_ function and records the time it takes

In [7]:

# Define your paths
isolated_dir = Path("../FileReadingStoring/MLAmorphousIsolatedExperiment").resolve()
data_dir = Path("../FileReadingStoring/AmorphousMLDataBase/").resolve()

os.makedirs(long_path(isolated_dir), exist_ok=True)
os.makedirs(long_path(data_dir), exist_ok=True)
Correction = True

# Move all files back in case some files got left behind on the isolated folder
for file_path in isolated_dir.iterdir():  # iterates Path objects
    if file_path.is_file():  # only move files
        dst = data_dir / file_path.name
        shutil.move(long_path(file_path), long_path(dst))
        log_message(f"Moved {file_path.name} back to {data_dir}")

for Complexity in ["Simple", "Naif", "Naif834", "Naif838", "Naif8316", "Naif8332", "Naif854", "Naif858", "Naif8516", "Naif8532", "Naif8104", "Naif8108", "Naif81016", "Naif81032", "Naif1634", "Naif1638", "Naif16316", "Naif16332", "Naif1654", "NaifTwice1D483316", "NaifTwice1D483332", "NaifTwice1D483516", "NaifTwice1D483532", "NaifTwice1D485316", "NaifTwice1D485332", "NaifTwice1D485516", "NaifTwice1D485532", "NaifTwice1D443316", "NaifTwice1D443332", "NaifTwice1D443516", "NaifTwice1D443532", "NaifTwice1D445316", "NaifTwice1D445332", "NaifTwice1D445516", "NaifTwice1D445532", "NaifTwiceDense414", "NaifTwiceDense418", "NaifTwiceDense4116", "NaifTwiceDense4124", "NaifTwiceDense814", "NaifTwiceDense818", "NaifTwiceDense8116", "NaifTwiceDense8124", "NaifTwiceDense1614", "NaifTwiceDense1618", "NaifTwiceDense16116", "NaifTwiceDense16124", "NaifTwiceDense2414", "NaifTwiceDense2418", "NaifTwiceDense24116", "NaifTwiceDense24124", "SimpleNoDropout888", "SimpleNoDropout8164", "SimpleNoDropout8168", "SimpleNoDropout1644", "SimpleNoDropout1648", "SimpleNoDropout1684", "SimpleNoDropout1688", "SimpleNoDropout16164", "SimpleNoDropout16168"]:
    build_model = Define_Complexity(Complexity)
    log_message(f"Begin tests with model type {Complexity}")
    print({Complexity})
    for num_augmentations in [5]:  # idem with augmentations
        log_message(f"Begin {Complexity} model with {num_augmentations} augmentations")
        # Delete folder if exists
        for file_path in isolated_dir.iterdir():  # iterates Path objects
            if file_path.is_file():  # only move files
                dst = data_dir / file_path.name
                shutil.move(long_path(file_path), long_path(dst))
                log_message(f"  Moved {file_path.name} back to {data_dir}")

        # Path to the log file
        log_file = Path("AmorphousExecution_times.txt")

        # Start timing
        start_time = time.time()
        folder_to_delete = Path(f"AmorphousAllTestsFolder_{Complexity}_{num_augmentations}").resolve()
        if folder_to_delete.exists() and folder_to_delete.is_dir():
            shutil.rmtree(long_path(folder_to_delete))
            log_message(f"Deleted folder: {folder_to_delete}")
        else:
            log_message(f"Folder does not exist: {folder_to_delete}")
        output_folder = Path("..") / "ML" / "Tests" / f"AmorphousAllTestsFolder_{Complexity}_{num_augmentations}"
        output_folder.mkdir(parents=True, exist_ok=True)

        use_uncertainty = True
        
        isolate_experiments(
            long_path(data_dir),
            long_path(isolated_dir),
            long_path(output_folder),
            use_uncertainty,
            num_augmentations,
            Correction=Correction,
            build_model=build_model
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        with open(long_path(log_file), "a") as f:  # append mode
            f.write(f"Execution time: {elapsed_time:.6f} seconds\n")
        log_message(f"Execution finished in {elapsed_time:.6f} seconds. Logged to {log_file}")
        log_message(f"\n _______________________________________________________ \n ")

{'Simple'}


NameError: name 'psutil' is not defined